### Introduction

Here we are going to build a collaborative filtering recommendation engine which relies on past purchases of a customer. In simple words, we make recommendations based on, what all products have been most commonly brought along with the product purchased by the customer. 

It's not the most powerful approach, but should give you a quick lead on solving this problem.


In [2]:
## load libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import scipy.sparse as sps
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [7]:
# load product and sample file
products = pd.read_csv('../products.csv')

In [8]:
sample = pd.read_csv('../sampleSubmission.csv')

In [9]:
sample.head()

,customerID,products
0,BBID_204221,"300663432,1000099534,1000475598,None,None,None..."
1,BBID_204254,"300663432,1000099534,1000475598,None,None,None..."
2,BBID_204830,"300663432,1000099534,1000475598,None,None,None..."
3,BBID_204880,"300663432,1000099534,1000475598,None,None,None..."
4,BBID_204910,"300663432,1000099534,1000475598,None,None,None..."


In [10]:
'BBID_20410043' in sample['customerID']

False

In [11]:
## number of unique customers
products['customerID'].nunique()

165055

### To avoid memory error, lets build first model using June 2016 onwards data

In [12]:
products['transactionDate'] = pd.to_datetime(products['transactionDate'])
mask = (products['transactionDate'] >= '2016-12-01')
products = products.loc[mask]

In [13]:
products.sort_values('transactionDate',inplace=True)
products = products.reset_index(drop=True)

In [14]:
## take only those customers which are in sample submission file
products_2 = products[products['customerID'].isin(sample['customerID'])]

## remove missing values # 4
products_2 = products_2[~pd.isnull(products_2['product_code'])]

## convert type of product code
products_2['product_code'] = products_2['product_code'].astype(np.int64)

In [15]:
products_2 = products_2.loc[:,['customerID','product_code']]
products_2 = products_2.reset_index(drop=True)

In [16]:
## these customers are not in train, so we'll predict None for them at last
misfit_customers = list(set(sample['customerID']) - set(products_2['customerID']))

### user X product matrix

In [17]:
## create product list by customers
products_2 = products_2.groupby('customerID')['product_code'].apply(lambda x: x.tolist()).reset_index()

In [18]:
## remove duplicate products
products_2['product_code'] = products_2['product_code'].map(lambda x: list(set(x)))

In [19]:
## fix product max len to 20 (we'll pick the last 20 i.e most recent ones)
products_2['product_code'] = products_2['product_code'].map(lambda x: x[-20:])

In [20]:
### create a list of customers & products

customerIDs = []
product_codes = []

for index, row in products_2.iterrows():
    #if index % 10 == 0:
    #    print (index)
    ls_len = len(row['product_code'])
    customerIDs.extend(np.repeat(row['customerID'], ls_len))
    product_codes.extend(row['product_code'])

In [21]:
## encode values 

from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder() ## for customers
customerIDs = lbl.fit_transform(customerIDs)
 
lbl2 = LabelEncoder() ## for products
product_codes = lbl2.fit_transform(product_codes)


In [22]:
n_unique_users = len(set(customerIDs))
n_unique_products = len(set(product_codes))

row = customerIDs
col = product_codes

vals = np.repeat(1, len(row))

## this matrix has information about a user bought which all products.
user_product_matrix = sps.csr_matrix((vals, (row, col)), shape=(n_unique_users, n_unique_products))

## this matrix has information a product got bought along with which other products, and how many times
product_cooccurence_matrix = (user_product_matrix.T * user_product_matrix)

## set diagonal equals to zero since we are not interested in knowing the count of a product with itself
product_cooccurence_matrix.setdiag(np.repeat(0, product_cooccurence_matrix.shape[0]))

In [23]:
## create a data frame of encoded values
product_summary = pd.DataFrame({'customerID':customerIDs, 'product_code':product_codes})
product_summary = product_summary.groupby('customerID')['product_code'].agg(lambda x:x.tolist()).reset_index().rename(columns = {0:'product_collection'})

In [24]:
## how many recommendation to make
def take_top_(x):
    if x >= 20:
        return 1
    else:
        if x < 20:
            return int(np.round(20/x))
        
## get count of products per customer
product_summary['len_collection'] = product_summary['product_collection'].map(len)

## if a customer has 20 products in the list, we'll take 1 top most product per each product
product_summary['take_top'] = product_summary['len_collection'].map(lambda x: take_top_(x))

In [25]:
## recommendation function
def recommend_affinity(user):
    
    products_ = product_summary[product_summary['customerID'] == user]['product_collection'].iloc[0] # returns list
    take_top_ = product_summary[product_summary['customerID'] == user]['take_top'].iloc[0]
    
    recs = []
    
    if take_top_ == 0:
        return recs
    
    ## here we get the list of products which were bought the maximum number of times along with a particular productr
    for tt in products_:
        s = np.squeeze(np.asarray(product_cooccurence_matrix[tt].todense())) ## list of products bought with counts` with that product
        ll = s.argsort()[-int(take_top_):][::-1]
        recs.append(list(ll))

    recs = list(np.vstack(recs).flatten('F'))
 
    if not recs:
        return recs
    
    return recs

In [26]:
## recommendation for customers
unique_customers = []

for i in sample['customerID']:
    if i not in misfit_customers:
        unique_customers.append(i)

unique_customers = lbl.fit_transform(unique_customers)

In [27]:
from collections import defaultdict
out_dict = defaultdict(list)

nulls = []

for user in tqdm(unique_customers): #[:50]): ## i took first 50 customers. 
    rec = recommend_affinity(user)
    if not rec:
        nulls.append(user)
    out_dict[user] = rec
#     if i% 10 == 0:
#         print(i)

100%|██████████| 25754/25754 [3:06:12<00:00,  2.31it/s]  


In [29]:
## get unique items per customer
from collections import defaultdict

out_dict_2 = defaultdict(list)

for k,v in out_dict.items():
    out_dict_2[k] = list(set(v))

In [30]:
## convert key values to inverse customer codes

customer_codes = list(lbl.inverse_transform(customerIDs))
customer_maps = dict(zip(list(customerIDs), list(customer_codes)))

In [31]:
out_dict_3 = defaultdict(list)

for k,v in out_dict_2.items():
    out_dict_3[customer_maps[k]] = out_dict_2[k]

In [32]:
mis_dict = defaultdict(list)

for i in misfit_customers:
    mis_dict[i] = list(np.repeat('None', 20))

In [33]:
out_dict_3.update(mis_dict)

In [34]:
submission = pd.DataFrame(list(out_dict_3.items()), columns=['customerID','products'])

In [35]:
submission.head()

,customerID,products
0,BBID_211417787,"[6105, 5916, 5917, 6183]"
1,BBID_211417786,"[8434, 5916, 5917, 1806, 6183]"
2,BBID_204110903,"[6640, 2340, 5916, 5917, 6183]"
3,BBID_204110902,"[None, None, None, None, None, None, None, Non..."
4,BBID_211417783,"[1927, 6183, 5714, 5915, 5916, 5917]"


In [36]:
product_codes_inv = list(lbl2.inverse_transform(product_codes))
product_maps = dict(zip(list(product_codes), list(product_codes_inv)))

In [37]:
submission['products'] = submission['products'].map(lambda x: [product_maps.get(s) for s in x])

In [42]:
for index, row in submission.iterrows():
    max_len = 20
    if len(row['products']) < max_len:
        size = len(row['products'])
        diff_ = max_len - size
        a = row['products']
        a.extend(np.repeat('None', diff_))
        submission.loc[index, 'products'] = a
    else:
        submission.loc[index, 'products'] = row['products'][:20]

In [43]:
submission['products'] = submission['products'].map(lambda x: ','.join(str(e) for e in x))

In [44]:
submission.head()

,customerID,products
0,BBID_211417787,"3,0,0,8,2,5,5,9,6,,,3,0,0,7,7,6,4,1,0,,"
1,BBID_211417786,"1,0,0,0,0,3,8,5,2,2,,,3,0,0,7,7,6,4,1,0"
2,BBID_204110903,"3,0,0,9,4,2,6,9,7,,,1,0,8,0,3,5,4,3,6,,"
3,BBID_204110902,"N,o,n,e,,,N,o,n,e,,,N,o,n,e,,,N,o,n,e,,"
4,BBID_211417783,"1,0,8,0,2,7,3,4,8,,,3,0,0,8,4,0,0,1,8,,"


In [46]:
submission.to_csv('../sub_0001.csv', index=False)